In [1]:
import requests
import bs4 as bs
import json
import pandas as pd

url ="https://api.yelp.com/v3/businesses/search"


In [2]:

api_key='T_QRoPeEzUgm8yWqddvVOSKLeGgjS8AeGty1lFZNnmVqnrrHANzld2ouQJjTYqcllg2ApKmCq5WprTw-iKYrQXf3Uxp218Zu6R3F-DW19HCOv-lWqG8NF-JacIajY3Yx'
headers = {'Authorization': 'Bearer %s' % api_key }


In [3]:
def get_businesses(location, term):
    headers = {'Authorization': 'Bearer %s' % api_key}
    url = 'https://api.yelp.com/v3/businesses/search'

    data = []
    
    for offset in range(0, 1000, 50):
        params = {
            'limit': 50, 
            'location': location.replace(' ', '+'),
            'term': term.replace(' ', '+'),
            'offset': offset
        }

        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            data += response.json()['businesses']
        elif response.status_code == 400:
            print('400 Bad Request')
            break
     #cleaning up the data       
    df = pd.DataFrame(data)
    df = df[['name', 'review_count', 'rating', 'categories', 'transactions', 'price', 'coordinates', 'id', 'location' ]] #keeping only necesssary columns
    df['category'] = [[d.get('title') for d in x] for x in df['categories']] #unpacking list of dicts and keeping title values
    df.drop(['categories'], axis = 1, inplace = True)
    df['category'] = df['category'].str.get(0) #only keeping the first title value
    df['delivery'] = df.transactions.map(set(['delivery']).issubset) #creating separate column indicating whether place offers delivery
    df['pickup'] = df.transactions.map(set(['pickup']).issubset)#creating sep column indicating whether place offers pickup
    df['latitude'] = [x.get('latitude') for x in df['coordinates']] #creating sep column for lat
    df['longitude'] = [x.get('longitude') for x in df['coordinates']] #creating dep column for long
    df['city'] = [x.get('city') for x in df['location']]
    df.drop(['transactions', 'coordinates', 'location'], axis = 1, inplace = True)

    return df

In [4]:
df1 = get_businesses('Los Angeles', 'Boba')

In [5]:
df1

,name,review_count,rating,price,id,category,delivery,pickup,latitude,longitude,city
0,Heuk Hwa Dang,139,4.0,$$,QLwjx7iCGaC7EOjNI3kxlg,Bubble Tea,True,True,34.062800,-118.309280,Los Angeles
1,Creek Tea,72,4.5,$$,uB1BUlhx7nzLa4_LUnmSDw,Ice Cream & Frozen Yogurt,True,True,34.063992,-118.309589,Los Angeles
2,Boba Bear,1001,4.0,$,QUK6uKjFBN1HWCpxwC0oMg,Juice Bars & Smoothies,True,True,34.066600,-118.308700,Los Angeles
3,Boba Zzing,88,4.5,$$,-H_rg048Yj-LmrXbc_cd3Q,Bubble Tea,True,True,34.087776,-118.308660,Los Angeles
4,Yifang Taiwan Fruit Tea,266,4.5,$,b6IyTzXdRzDwm-ojVvZPiw,Bubble Tea,True,True,34.063388,-118.303698,Los Angeles
...,...,...,...,...,...,...,...,...,...,...,...
995,Melo Melo Coconut Dessert,380,4.0,$$,BMd8f8_TD8eICQYp0hdmeA,Desserts,True,True,34.080427,-118.093021,San Gabriel
996,Pho Saigon,420,3.5,$$,chg55EplntXfa72g6PDfHA,Vietnamese,True,True,34.063675,-118.306376,Los Angeles
997,Vim Thai-Chinese Restaurant,213,3.5,$$,21TIwq-BTC8O1BRx4CESew,Thai,True,False,34.101642,-118.301684,Los Angeles
998,Tub Tim Siam,204,4.0,$$,L2yfgrEDl_9D2sCB4BrnPg,Thai,True,True,34.103926,-118.287210,Los Angeles


In [6]:
sum(df1.duplicated())

0

In [7]:
df1['category'].value_counts()

Bubble Tea                   275
Coffee & Tea                 227
Desserts                      68
Donuts                        52
Ice Cream & Frozen Yogurt     40
                            ... 
Shanghainese                   1
Health Markets                 1
Coffee Roasteries              1
Candy Stores                   1
Spanish                        1
Name: category, Length: 69, dtype: int64

In [8]:
#for the sake of our dataset, let's only keep places that have category values greater than 8
# temp = df1['category'].tolist()
# df1 = df1[df1['category'].apply(lambda x: temp.count(x) > 35)]
# df1['category'].value_counts()

In [9]:
df1.rename(columns = {'id':'business_id'}, inplace = True)

In [10]:
df1.to_csv('yelp_ww_boba.csv', index = False)

In [11]:
reviews = []
from yelpapi import YelpAPI
yelp_api =  yelp_api = YelpAPI(api_key)
ids = df1['business_id']
for business_id in ids:
    try:
        response = yelp_api.reviews_query(business_id)["reviews"]
    except Exception as exc:
        print(f"Failed retrieving {business_id}: {exc}")
        # TODO: maybe retry this fetch?
        continue
    for review in response:
        # Merge the business ID into the dict, append to a list.
        reviews.append({**review, "business_id": business_id})

In [12]:
df_reviews = pd.DataFrame(reviews)
df_reviews.drop(['id', 'url', 'user'], axis = 1,inplace = True)
df_reviews.rename(columns = {'rating':'review_rating'}, inplace = True)
df_reviews


,text,review_rating,time_created,business_id
0,Needed to kill some time with a girlfriend in ...,5,2023-01-08 20:51:05,QLwjx7iCGaC7EOjNI3kxlg
1,Such a big location for a boba/croifle spot! I...,4,2023-01-06 20:22:44,QLwjx7iCGaC7EOjNI3kxlg
2,"Very busy, cute cafe with a trendy crowd. The ...",4,2022-12-27 17:17:32,QLwjx7iCGaC7EOjNI3kxlg
3,Beautiful and delicious Milk Tea!\n\nI got the...,5,2022-12-31 13:16:04,uB1BUlhx7nzLa4_LUnmSDw
4,New boba shop in LA! Everything is self order ...,4,2023-01-01 14:40:58,uB1BUlhx7nzLa4_LUnmSDw
...,...,...,...,...
2972,Been coming to this place for many years. As ...,4,2022-10-23 06:52:27,L2yfgrEDl_9D2sCB4BrnPg
2973,"Unpretentious, this place feels like home. \n ...",5,2022-03-01 20:26:11,L2yfgrEDl_9D2sCB4BrnPg
2974,I believe the last time we were here was 10yea...,5,2022-12-12 23:23:24,cZloVOidS9pc3icXU7BiRw
2975,Do NOT eat at Dazzle Han.\n\nIf I get food poi...,1,2022-12-13 18:46:29,cZloVOidS9pc3icXU7BiRw


In [13]:
df = pd.merge(df1, df_reviews, on="business_id")
df.drop(['city'], axis =1, inplace = True)
df

,name,review_count,rating,price,business_id,category,delivery,pickup,latitude,longitude,text,review_rating,time_created
0,Heuk Hwa Dang,139,4.0,$$,QLwjx7iCGaC7EOjNI3kxlg,Bubble Tea,True,True,34.062800,-118.309280,Needed to kill some time with a girlfriend in ...,5,2023-01-08 20:51:05
1,Heuk Hwa Dang,139,4.0,$$,QLwjx7iCGaC7EOjNI3kxlg,Bubble Tea,True,True,34.062800,-118.309280,Such a big location for a boba/croifle spot! I...,4,2023-01-06 20:22:44
2,Heuk Hwa Dang,139,4.0,$$,QLwjx7iCGaC7EOjNI3kxlg,Bubble Tea,True,True,34.062800,-118.309280,"Very busy, cute cafe with a trendy crowd. The ...",4,2022-12-27 17:17:32
3,Creek Tea,72,4.5,$$,uB1BUlhx7nzLa4_LUnmSDw,Ice Cream & Frozen Yogurt,True,True,34.063992,-118.309589,Beautiful and delicious Milk Tea!\n\nI got the...,5,2022-12-31 13:16:04
4,Creek Tea,72,4.5,$$,uB1BUlhx7nzLa4_LUnmSDw,Ice Cream & Frozen Yogurt,True,True,34.063992,-118.309589,New boba shop in LA! Everything is self order ...,4,2023-01-01 14:40:58
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2972,Tub Tim Siam,204,4.0,$$,L2yfgrEDl_9D2sCB4BrnPg,Thai,True,True,34.103926,-118.287210,Been coming to this place for many years. As ...,4,2022-10-23 06:52:27
2973,Tub Tim Siam,204,4.0,$$,L2yfgrEDl_9D2sCB4BrnPg,Thai,True,True,34.103926,-118.287210,"Unpretentious, this place feels like home. \n ...",5,2022-03-01 20:26:11
2974,Koreatown Plaza Food Corner,101,4.0,$$,cZloVOidS9pc3icXU7BiRw,Korean,False,False,34.055140,-118.308660,I believe the last time we were here was 10yea...,5,2022-12-12 23:23:24
2975,Koreatown Plaza Food Corner,101,4.0,$$,cZloVOidS9pc3icXU7BiRw,Korean,False,False,34.055140,-118.308660,Do NOT eat at Dazzle Han.\n\nIf I get food poi...,1,2022-12-13 18:46:29


In [14]:
df_reviews.to_csv('reviews.csv', index = False)